# Sequential Minimal Optimization

we will use an algorithm specifically created to solve this problem quickly: the SMO (sequential minimal optimization) algorithm. Most machine learning libraries use the SMO algorithm or some variation. 

The SMO algorithm will solve the following optimization problem:

$$
\begin{array}{cl}{\underset{\alpha}{\operatorname{minimize}}} & {\frac{1}{2} \sum_{i=1}^{m} \sum_{j=1}^{m} \alpha_{i} \alpha_{j} y_{i} y_{j} K\left(\mathbf{x}_{i}, \mathbf{x}_{j}\right)-\sum_{i=1}^{m} \alpha_{i}} \\ {\text { subject to }} & {0 \leq \alpha_{i} \leq C, \text { for any } i=1, \ldots, m} \\ {} & {\sum_{i=1}^{m} \alpha_{i} y_{i}=0}\end{array}
$$

It is a kernelized version of the soft-margin formulation 


In [3]:
def kernel(x1, x2):
    return np.dot(x1, x2.T) 
 
def objective_function_to_minimize(X, y, a, kernel):
    m, n = np.shape(X)
    cal = (1 / 2) * np.sum( [a[i] * a[j] * y[i] * y[j] * kernel(X[i, :], X[j, :]) for j in range(m) for i in range(m)] ) \
                        - np.sum(a)
    return cal

In [4]:
def kernel(x1, x2):
    return np.dot(x1, x2.T) 
 
def objective_function_to_minimize(X, y, a, kernel):
    m, n = np.shape(X)
    return 1 / 2 * np.sum([a[i] * a[j] * y[i] * y[j]* kernel(X[i, :], X[j, :])
                           for j in range(m)
                           for i in range(m)])\
                - np.sum([a[i] for i in range(m)]) 

This is the same problem we solved using CVXOPT. Why do we need another method? Because we would like to be able to use SVMs with big datasets, and using convex optimization packages usually involves matrix operations that take a lot of time as the size of the matrix increases or become impossible because of memory limitations. The SMO algorithm has been created with the goal of being faster than other methods

# The idea behind SMO 

The idea behind SMO is quite easy: we will solve a simpler problem. That is, given a vector $\alpha=\left(\alpha_{1}, \alpha_{2}, \ldots, \alpha_{m}\right)$, we will allow ourselves to change only two values of $\alpha$, for instance, $\alpha_3$ and $\alpha_7$. We will change them until the objective function reaches its minimum given this set of alphas. Then we will pick two other alphas and change them until the function returns its smallest value, and so on. If we continue doing that, we will eventually reach the minimum of the objective function of the original problem. 
SMO solves a sequence of several simpler optimization problems. 

### The SMO algorithm is composed of three parts: 

• One heuristic to choose the first Lagrange multiplier 

• One heuristic to choose the second Lagrange multiplier 

• The code to solve the optimization problem analytically for the two chosen multipliers 

In [5]:
def get_non_bound_indexes(self):
    return np.where(np.logical_and(self.alphas > 0,
                                   self.alphas < self.C))[0] 
 
# First heuristic: loop  over examples where alpha is not 0 and not C 
# they are the most likely to violate the KKT conditions 
# (the non-bound subset). 
def first_heuristic(self):
    num_changed = 0
    non_bound_idx = self.get_non_bound_indexes() 
    for i in non_bound_idx:
        num_changed += self.examine_example(i)
        return num_changed 

In [7]:
def main_routine(self):
    num_changed = 0
    examine_all = True 
 
    while num_changed > 0 or examine_all:
        num_changed = 0 
        if examine_all:
            for i in range(self.m):
                num_changed += self.examine_example(i)
        else:
            num_changed += self.first_heuristic() 
 
        if examine_all:
            examine_all = False
        elif num_changed == 0:
            examine_all = True

In [14]:
def second_heuristic(self, non_bound_indices):
    i1 = -1
    if len(non_bound_indices) > 1:
        maxi = 0 
    for j in non_bound_indices:
        E1 = self.errors[j] - self.y[j]
        step = abs(E1 - self.E2)# approximation
        if step > max:
            maxi = step
            i1 = j
    return i1 
 

def examine_example(self, i2):
    self.y2 = self.y[i2]
    self.a2 = self.alphas[i2]
    self.X2 = self.X[i2]
    self.E2 = self.get_error(i2) 
 
    r2 = self.E2 * self.y2 
 
    if not((r2 < -self.tol and self.a2 < self.C) or (r2 > self.tol and self.a2 > 0)):
        # The KKT conditions are met, SMO looks at another example.
        return 0 
 
    # Second heuristic A: choose the Lagrange multiplier that     
    # maximizes the absolute error.
    non_bound_idx = list(self.get_non_bound_indexes())
    i1 = self.second_heuristic(non_bound_idx) 
 
    if i1 >= 0 and self.take_step(i1, i2):
        return 1 
 
    # Second heuristic B: Look for examples making positive
    # progress by looping over all non-zero and non-C alpha,
    # starting at a random point.
    if len(non_bound_idx) > 0:
        rand_i = randrange(len(non_bound_idx))
        for i1 in (non_bound_idx[rand_i:] + non_bound_idx[:rand_i]):
            if self.take_step(i1, i2):
                return 1 
 
    # Second heuristic C: Look for examples making positive progress
    # by looping over all possible examples, starting at a random 
    # point.
    rand_i = randrange(self.m)
    all_indices = list(range(self.m))
    for i1 in all_indices[rand_i:] + all_indices[:rand_i]:
        if self.take_step(i1, i2):
            return 1 
 
    # Extremely degenerate circumstances, SMO skips the first example.     return 0